## TORCH.JIT.FREEZE

<b>
    
```python
torch.jit.freeze(mod, preserved_attrs=None, optimize_numerics=True)
```
<b>

Freezing a ```ScriptModule``` will clone it and attempt to inline the cloned module's submodules, parameters and 
attributes as constant in the TorchScript IR Graph. By default, <i>```forward```</i> will be preserved, as well as attributes & methods specified in ```preserved_attrs```. Additionally, any attribute that is modified within a preserved method will be preserved.

Freezing currently only accepts ```ScriptModules``` that are in ```eval``` mode.

Freezing applies generic optimization that will speed up your model regardless of machine. To further optimize using server-specific settings, run <b><i>```optimize_for_inference```</i></b> after freezing.


### Parameters
- <b>mod</b> (```ScriptModule```) := A module to be frozen
- <b>preserved_attrs</b> (```Optional[List[str]]```) := A list of attributes to preserve addition to the forward method.
- <b>optimize_numerics</b> (```bool```) := If ```True```, a set of optimization passes will be run that does not strictly.

### Returns
Frozen ```ScriptModule```

### Example (Freezing a simple module with a Parameter) 

In [1]:
import torch

class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        
        self.weight = torch.nn.Parameter(torch.randn(N, M))
        self.linear = torch.nn.Linear(N, M)
    
    def forward(self, input):
        output = self.weight.mm(input)
        output = self.linear(output)
        return output
    
scripted_module = torch.jit.script(MyModule(2, 3).eval())
frozen_module = torch.jit.freeze(scripted_module)

# parameters have been removed and inlined into the Graph as constants
assert len(list(frozen_module.named_parameters())) == 0
# See the compiled graph as Python code
print(frozen_module.code)
print('Graph: ', frozen_module.graph)

def forward(self,
    input: Tensor) -> Tensor:
  output = torch.mm(CONSTANTS.c0, input)
  if torch.eq(torch.dim(output), 2):
    ret = torch.addmm(CONSTANTS.c1, output, CONSTANTS.c2, beta=1, alpha=1)
    output0 = ret
  else:
    output1 = torch.matmul(output, CONSTANTS.c2)
    output2 = torch.add_(output1, CONSTANTS.c1, alpha=1)
    output0 = output2
  return output0

Graph:  graph(%self : __torch__.___torch_mangle_0.MyModule,
      %input.1 : Tensor):
  %22 : Float(2:1, 3:2, requires_grad=0, device=cpu) = prim::Constant[value= 0.3563 -0.5785  0.5724 -0.3773 -0.4578  0.3807 [ CPUFloatType{2,3} ]]()
  %7 : int = prim::Constant[value=2]() # C:\ProgramData\Anaconda3\envs\ML\lib\site-packages\torch\nn\functional.py:1688:22
  %6 : int = prim::Constant[value=1]()
  %self.linear.bias : Float(3:1, requires_grad=0, device=cpu) = prim::Constant[value=-0.0949  0.2828  0.6798 [ CPUFloatType{3} ]]()
  %self.weight : Float(2:3, 3:1, requires_grad=0, device=cpu) = prim::Constant[value=-0.5692  0.20

### Example (Freezing a Module with preserved attributes)

In [4]:
import torch

class MyModule2(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        
    def forward(self, input):
        self.modified_tensor += 1
        return input + self.modified_tensor
    

scripted_module = torch.jit.script(MyModule2().eval())
frozen_module = torch.jit.freeze(scripted_module, preserved_attrs=["version"])
# we've manually preserved `version`, so it still exists on the frozen module and can be modified
assert frozen_module.version == 1
frozen_module.version = 2
# `modified_tensor` is detected as being mutated in the forward, so freezing preserves
# it to retain model semantics
assert frozen_module(torch.tensor(1)) == torch.tensor(12)
# now that we've run it once, the next result will be incremented by one
assert frozen_module(torch.tensor(1)) == torch.tensor(13)

TypeError: super(type, obj): obj must be an instance or subtype of type